### Week 8 Homework

#### Q1
$$
\begin{split}
    & \eta(\tau,\delta) - \sum_{k=0}^\delta \eta(\tau-1,k) \\
    = & \frac{(\tau + \delta)!}{\tau!~\delta!} - \sum_{k=0}^\delta \frac{(\tau - 1 + k)!}{(\tau - 1)!~ k!} \\
    = & \frac{(\tau + \delta)!}{\tau!~\delta!} - \sum_{k=0}^\delta \frac{\tau~(\tau - 1 + k)!}{(\tau)!~ k!} \\
    = & \frac{1}{\tau !} \left(
            \frac{(\tau + \delta)!}{\delta!} - \frac{\tau~(\tau - 1 + \delta)!}{\delta!} - \sum_{k=0}^{\delta - 1} \frac{\tau~(\tau - 1 + k)!}{k!}
        \right)\\
    = & \frac{1}{\tau !} \left(
            \frac{(\tau + \delta - \delta)(\tau + \delta - 1)!}{(\delta)!} - \sum_{k=0}^{\delta - 1} \frac{\tau~(\tau - 1 + k)!}{k!}
        \right)\\
    = & \frac{1}{\tau !} \left(
            \frac{(\tau + \delta - 1)!}{(\delta - 1)!} - \sum_{k=0}^{\delta - 1} \frac{\tau~(\tau - 1 + k)!}{k!}
        \right)\\
     = & \eta(\tau,\delta - 1) - \sum_{k=0}^{\delta - 1} \eta(\tau-1,k)\\
     = & \eta(\tau,0) - \eta(\tau-1,0) = 1 - 1 = 0
\end{split}
$$

#### Tasks

Calculate the square root of norm.

In [14]:
using ForwardDiff
using Test

function poorman_norm(x::Vector{<:Real})
	nm2 = zero(real(eltype(x)))
	for i=1:length(x)
		nm2 += abs2(x[i])
	end
	ret = sqrt(nm2)
	return ret
end

poorman_norm (generic function with 1 method)

##### Task1
For forward autodiff

In [15]:
function forward_diff_norm!(x::Vector{<:Real}, partial_x::Vector{<:Real})
    nm2 = zero(real(eltype(x)))
    for i = 1:length(x)
        nm2 += abs2(x[i])
        partial_x[i] *= 2 * x[i]
    end
    ret = sqrt(nm2)
    partial_x .*= 1 / (2 * ret)
    return ret, partial_x
end

forward_diff_norm! (generic function with 1 method)

In [16]:
@testset "Forward mode autodiff test" begin
    for i in 1:100
        N = rand(1:100)
        x = rand(N)
        partial_x = [1.0 for i in 1:N]
        ret, diff_x = forward_diff_norm!(x, partial_x)
        @test ret ≈ poorman_norm(x)
        @test diff_x ≈ ForwardDiff.gradient(poorman_norm, x)        
    end
end

Test Summary:              | Pass  Total  Time
Forward mode autodiff test |  200    200  0.3s


Test.DefaultTestSet("Forward mode autodiff test", Any[], 200, false, false, true, 1.684315906093323e9, 1.68431590637698e9)

##### Task2

This code uses a stack to store $N+1$ data ($x$ is a $N-d$ data)

In [17]:
# Forward process
function reverse_norm_forward!(x::Vector{<:Real}, stack::Vector{<:Real})
    nm2 = zero(real(eltype(x)))
	for i=1:length(x)
		push!(stack, x[i])
		nm2 += abs2(x[i])
	end
	push!(stack, nm2)
	ret = sqrt(nm2)
	return ret, stack
end

# Backward process
function reverse_norm_backward!(partial_x::Vector{<:Real}, stack::Vector{<:Real})
	nm2 = pop!(stack)
	partial_x .*= 1 / (2 * sqrt(nm2))
	N = length(partial_x)
	for i in 1:N
		x = pop!(stack)
		partial_x[N + 1 - i] *= 2 * x
	end
	return partial_x, stack
end

reverse_norm_backward! (generic function with 1 method)

In [19]:
@testset "Reverse mode autodiff test" begin
    for i in 1:100
        N = rand(1:100)
        x = rand(N)
        partial_x = [1.0 for i in 1:N]
        stack = Vector{Real}()
        ret, stack = reverse_norm_forward!(x, stack)
        partial_x, stack = reverse_norm_backward!(partial_x, stack)
        @test ret ≈ poorman_norm(x)
        @test partial_x ≈ ForwardDiff.gradient(poorman_norm, x)        
    end
end

Test Summary:              | Pass  Total  Time
Reverse mode autodiff test |  200    200  0.1s


Test.DefaultTestSet("Reverse mode autodiff test", Any[], 200, false, false, true, 1.684315926708938e9, 1.68431592679219e9)

#### Credits to Xuanzhao Gao